<a href="https://colab.research.google.com/github/faezehhyd/NLP_Assignment_1/blob/main/NLP_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
from nltk import classify
from nltk import NaiveBayesClassifier
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.stem import PorterStemmer, SnowballStemmer, WordNetLemmatizer
from nltk import NaiveBayesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report


In [2]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
!pip install wikipedia-api

In [4]:
# Keywords for each category
category1_keywords = ["region", "terrain", "map", "nation", "urban"]
category2_keywords = ["radio", "frequency", "bandwidth", "satellite", "mobile", "sector", "network"]

# 'Geographic' and 'Non-geographic'
Geo_category = "Geographical"
Nongeo_category = "Nongeographical"

In [5]:
import wikipediaapi

def retrieve_wikipedia_content_with_markers(subject, keywords):
    # A Wikipedia API object creation
    wiki_instance = wikipediaapi.Wikipedia('en', extract_format=wikipediaapi.ExtractFormat.WIKI, headers={'User-Agent': 'Faezeh'})

    # Assigning  Wikipedia page based on a subject
    page = wiki_instance.page(subject)

    if page.exists():
        # Retrieve the text content of the page
        content = page.text

        # Identify markers or keywords in the text
        markers = [kw for kw in keywords if kw.lower() in content.lower()]

        return content, markers
    else:
        print(f"Page for subject '{subject}' does not exist.")
        return None, None

# Get the content of Wikipedia page with markers
topic = "telecommunication"
text_content, markers_found = retrieve_wikipedia_content_with_markers(topic, category2_keywords)

if text_content:
    print(f"Text content for '{topic}':")
    print("Markers Found:", markers_found)


Text content for 'telecommunication':
Markers Found: ['radio', 'frequency', 'bandwidth', 'satellite', 'mobile', 'sector', 'network']


In [6]:
def process_text_with_annotations(input_text, input_annotations, stop_words=None, stemmer=None, lemmatizer=None):
    # Tokenize the input text
    words = word_tokenize(input_text)

    # Utilize Snowball stop words if available; otherwise, default to using NLTK's standard English stop words
    stop_words = stop_words or set(stopwords.words('english'))

    # Eliminate stop words
    filtered_words = [word for word in words if word.lower() not in stop_words]

    # If a Snowball stemmer is available, utilize it; otherwise, using the Porter Stemmer
    stemmer = stemmer or PorterStemmer()
    stemmed_words = [stemmer.stem(word) for word in filtered_words]

    # WordNet Lemmatizer available, then use it, if not use a basic one
    lemmatizer = lemmatizer or WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in stemmed_words]

    # Combine the processed words into a single string
    processed_text = ' '.join(lemmatized_words)

    # Integrate annotations into the processed text
    processed_text += ' '.join(input_annotations)

    return processed_text


# examples
subject = "telecommunication"

# Get Wikipedia content page with markers
text_content, markers_found = retrieve_wikipedia_content_with_markers(subject, category2_keywords)

if text_content:
    print(f"Text content for '{subject}':")
    print("Markers Found:", markers_found)

    # Process the text with different configurations
    processed_text_bag_of_words = text_content.lower()
    processed_text_snowball = process_text_with_annotations(text_content, markers_found, stop_words=set(stopwords.words('english')), stemmer=SnowballStemmer('english'))
    processed_text_lemmatized = process_text_with_annotations(text_content, markers_found, stop_words=set(stopwords.words('english')), lemmatizer=WordNetLemmatizer())


Text content for 'telecommunication':
Markers Found: ['radio', 'frequency', 'bandwidth', 'satellite', 'mobile', 'sector', 'network']


In [7]:
snowball_stemmer = SnowballStemmer('english')


# Training data for Logistic Regression with pre-processing
training_data_logistic_preprocessed_custom = [
    (process_text_with_annotations(retrieve_wikipedia_content_with_markers("Germany", category1_keywords)[0], category1_keywords, stop_words=set(stopwords.words('english')),
                                                                            stemmer=snowball_stemmer).lower(), Geo_category),
    (process_text_with_annotations(retrieve_wikipedia_content_with_markers("telecommunication", category2_keywords)[0], category2_keywords, stop_words=set(stopwords.words('english')),
                                                                            stemmer=snowball_stemmer).lower(), Nongeo_category),
]


# Tokenizer function for preprocessed text
def get_words_in_preprocessed_text_logistic_custom(text):
    return word_tokenize(text)

# Function for extracting features from preprocessed text
def extract_features_preprocessed_logistic_custom(text):
    return ' '.join(get_words_in_preprocessed_text_logistic_custom(text))

# Prepare the training set for Logistic Regression _ preprocessing
training_set_logistic_preprocessed_custom = [(extract_features_preprocessed_logistic_custom(text), label) for (text, label) in training_data_logistic_preprocessed_custom]

# Create TF-IDF vectors from the training set for Logistic Regression_ pre-processing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Separate features (X) and labels (y) for Logistic Regression with pre-processing
X_train_logistic_preprocessed_custom = [text for (text, _) in training_set_logistic_preprocessed_custom]
y_train_logistic_preprocessed_custom = [label for (_, label) in training_set_logistic_preprocessed_custom]

# Create TF-IDF vectors from the training set for Logistic Regression with pre-processing
tfidf_vectorizer_logistic_preprocessed_custom = TfidfVectorizer()
X_train_tfidf_logistic_preprocessed_custom = tfidf_vectorizer_logistic_preprocessed_custom.fit_transform(X_train_logistic_preprocessed_custom)

# Train the Logistic Regression classifier_ pre-processing
logistic_regression_classifier_preprocessed_custom = LogisticRegression()
logistic_regression_classifier_preprocessed_custom.fit(X_train_tfidf_logistic_preprocessed_custom, y_train_logistic_preprocessed_custom)


# Usage for Logistic Regression with pre-processing
test_text_logistic_preprocessed_custom = process_text_with_annotations(retrieve_wikipedia_content_with_markers("Germany", category1_keywords)[0], category1_keywords, stop_words=set(stopwords.words('english')), stemmer=snowball_stemmer).lower()
test_features_logistic_preprocessed_custom = tfidf_vectorizer_logistic_preprocessed_custom.transform([extract_features_preprocessed_logistic_custom(test_text_logistic_preprocessed_custom)])
classification_logistic_preprocessed_custom = logistic_regression_classifier_preprocessed_custom.predict(test_features_logistic_preprocessed_custom)

print(f"\nclass prediction for Logistic Regression with pre-processing: {classification_logistic_preprocessed_custom[0]} ")

test_text_logistic_preprocessed_custom = process_text_with_annotations(retrieve_wikipedia_content_with_markers("telecommunication", category2_keywords)[0], category2_keywords, stop_words=set(stopwords.words('english')), stemmer=snowball_stemmer).lower()
test_features_logistic_preprocessed_custom = tfidf_vectorizer_logistic_preprocessed_custom.transform([extract_features_preprocessed_logistic_custom(test_text_logistic_preprocessed_custom)])
classification_logistic_preprocessed_custom = logistic_regression_classifier_preprocessed_custom.predict(test_features_logistic_preprocessed_custom)

print(f"class prediction for Logistic Regression with pre-processing: {classification_logistic_preprocessed_custom[0]} ")

# Training data for Logistic Regression without pre-processing
training_data_logistic_raw_custom = [
    (retrieve_wikipedia_content_with_markers("Germany", category1_keywords)[0].lower(), Geo_category),
    (retrieve_wikipedia_content_with_markers("telecommunication", category2_keywords)[0].lower(), Nongeo_category),
]

# Tokenizer function for raw text
def get_words_in_raw_text_logistic_custom(text):
    return word_tokenize(text)

# Feature extraction function for raw text
def extract_features_raw_logistic_custom(text):
    return ' '.join(get_words_in_raw_text_logistic_custom(text))

# Prepare the training set for Logistic Regression without pre-processing
training_set_logistic_raw_custom = [(extract_features_raw_logistic_custom(text), label) for (text, label) in training_data_logistic_raw_custom]

# Create TF-IDF vectors from the training set for Logistic Regression without pre-processing
X_train_logistic_raw_custom = [text for (text, _) in training_set_logistic_raw_custom]
y_train_logistic_raw_custom = [label for (_, label) in training_set_logistic_raw_custom]

tfidf_vectorizer_logistic_raw_custom = TfidfVectorizer()
X_train_tfidf_logistic_raw_custom = tfidf_vectorizer_logistic_raw_custom.fit_transform(X_train_logistic_raw_custom)

# Train the Logistic Regression classifier without pre-processing
logistic_regression_classifier_raw_custom = LogisticRegression()
logistic_regression_classifier_raw_custom.fit(X_train_tfidf_logistic_raw_custom, y_train_logistic_raw_custom)

# Usage for Logistic Regression without pre-processing
test_text_logistic_raw_custom = retrieve_wikipedia_content_with_markers("Germany", category1_keywords)[0].lower()
test_features_logistic_raw_custom = tfidf_vectorizer_logistic_raw_custom.transform([extract_features_raw_logistic_custom(test_text_logistic_raw_custom)])
classification_logistic_raw_custom = logistic_regression_classifier_raw_custom.predict(test_features_logistic_raw_custom)

print(f"\nclass Prediction for Logistic Regression without pre-processing: {classification_logistic_raw_custom[0]} ")

test_text_logistic_raw_custom = retrieve_wikipedia_content_with_markers("Data Science", category2_keywords)[0].lower()
test_features_logistic_raw_custom = tfidf_vectorizer_logistic_raw_custom.transform([extract_features_raw_logistic_custom(test_text_logistic_raw_custom)])
classification_logistic_raw_custom = logistic_regression_classifier_raw_custom.predict(test_features_logistic_raw_custom)

print(f"class Prediction for Logistic Regression without pre-processing: {classification_logistic_raw_custom[0]} ")



class prediction for Logistic Regression with pre-processing: Geographical 
class prediction for Logistic Regression with pre-processing: Nongeographical 

class Prediction for Logistic Regression without pre-processing: Geographical 
class Prediction for Logistic Regression without pre-processing: Nongeographical 


In [8]:
'''Naive Bayes without Pre-processing'''

# Training data for Naive Bayes on Bag of Words without pre-processing
training_data_bow_raw_custom = [
    (retrieve_wikipedia_content_with_markers("Germany", Geo_category)[0].lower(), Geo_category),
    (retrieve_wikipedia_content_with_markers("telecommunication", Nongeo_category)[0].lower(), Nongeo_category),
]

# Tokenizer function for raw text
def get_words_in_raw_text_bow_custom(text):
    return word_tokenize(text)

# Feature extraction function for raw text
def extract_features_raw_bow_custom(text):
    return {word: True for word in get_words_in_raw_text_bow_custom(text)}

# Prepare the training set without pre-processing for Bag of Words
training_set_bow_raw_custom = [(extract_features_raw_bow_custom(text), label) for (text, label) in training_data_bow_raw_custom]

# Train the Naive Bayes classifier without pre-processing for Bag of Words
naive_bayes_classifier_bow_raw_custom = NaiveBayesClassifier.train(training_set_bow_raw_custom)

# Usage without pre-processing for Bag of Words
test_text_bow_raw_custom = retrieve_wikipedia_content_with_markers("Germany", Geo_category)[0].lower()
test_features_bow_raw_custom = extract_features_raw_bow_custom(test_text_bow_raw_custom)
classification_bow_raw_custom = naive_bayes_classifier_bow_raw_custom.classify(test_features_bow_raw_custom)

print(f"\nclass prediction for the test text using Bag of Words without pre-processing: {classification_bow_raw_custom}")



test_text_bow_raw_custom = retrieve_wikipedia_content_with_markers("telecommunication", Nongeo_category)[0].lower()
test_features_bow_raw_custom = extract_features_raw_bow_custom(test_text_bow_raw_custom)
classification_bow_raw_custom = naive_bayes_classifier_bow_raw_custom.classify(test_features_bow_raw_custom)

print(f"\nclass prediction for the test text using Bag of Words without pre-processing: {classification_bow_raw_custom}")



class prediction for the test text using Bag of Words without pre-processing: Geographical

class prediction for the test text using Bag of Words without pre-processing: Nongeographical


In [9]:
'''Naive Bayes with Pre-processing'''

snowball_stemmer = SnowballStemmer('english')

# Training data for Naive Bayes with Snowball stop words and Porter Stemmer
training_data_bow_snowball_custom = [
    (process_text_with_annotations(retrieve_wikipedia_content_with_markers("Germany", category1_keywords)[0], category1_keywords, stop_words=set(stopwords.words('english')),
                                   stemmer=snowball_stemmer).lower(), Geo_category),
    (process_text_with_annotations(retrieve_wikipedia_content_with_markers("telecommunication", category2_keywords)[0], category2_keywords, stop_words=set(stopwords.words('english')),
                                   stemmer=snowball_stemmer).lower(), Nongeo_category),
]


# Tokenizer function for preprocessed text
def get_words_in_preprocessed_text_bow_custom(text):
    return word_tokenize(text)

# Feature extraction function for preprocessed text
def extract_features_preprocessed_bow_custom(text):
    return {word: True for word in get_words_in_preprocessed_text_bow_custom(text)}

# Prepare the training set with Snowball stop words and Porter Stemmer for Bag of Words
training_set_bow_snowball_custom = [(extract_features_preprocessed_bow_custom(text), label) for (text, label) in training_data_bow_snowball_custom]

# Train the Naive Bayes classifier with Snowball stop words and Porter Stemmer for Bag of Words
naive_bayes_classifier_bow_snowball_custom = NaiveBayesClassifier.train(training_set_bow_snowball_custom)

# Snowball stop words and Porter Stemmer for Bag of Words
test_text_bow_snowball_custom = process_text_with_annotations(retrieve_wikipedia_content_with_markers("Germany", category1_keywords)[0], category1_keywords, stop_words=set(stopwords.words('english')), stemmer=snowball_stemmer).lower()
test_features_bow_snowball_custom = extract_features_preprocessed_bow_custom(test_text_bow_snowball_custom)
classification_bow_snowball_custom = naive_bayes_classifier_bow_snowball_custom.classify(test_features_bow_snowball_custom)

print(f"\nContent of the Predicted class for the preprocessed text with Snowball stop words and Porter Stemmer for Bag of Words: {classification_bow_snowball_custom}")

test_text_bow_snowball_custom = process_text_with_annotations(retrieve_wikipedia_content_with_markers("telecommunication", category2_keywords)[0], category2_keywords, stop_words=set(stopwords.words('english')), stemmer=snowball_stemmer).lower()
test_features_bow_snowball_custom = extract_features_preprocessed_bow_custom(test_text_bow_snowball_custom)
classification_bow_snowball_custom = naive_bayes_classifier_bow_snowball_custom.classify(test_features_bow_snowball_custom)

print(f"\nContent of the Predicted class for the preprocessed text with Snowball stop words and Porter Stemmer for Bag of Words: {classification_bow_snowball_custom}")



Content of the Predicted class for the preprocessed text with Snowball stop words and Porter Stemmer for Bag of Words: Geographical

Content of the Predicted class for the preprocessed text with Snowball stop words and Porter Stemmer for Bag of Words: Nongeographical
